In [1]:
import os
from os import path as osp
import sys
  
from tvm import relay, runtime
from tvm.relay import testing
import tvm
from tvm import te

OSError: libnvinfer.so.8: cannot open shared object file: No such file or directory

In [14]:
dshape = (1, 28, 28)
net, params = relay.testing.mlp.get_workload(batch_size=dshape[0],num_classes=1000, dtype="float32")

In [17]:
with tvm.transform.PassContext(opt_level=3):
    compiled_graph_lib = tvm.relay.build_module.build(net, "cuda", params=params)
from tvm.contrib.debugger import debug_executor as graph_runtime
dev = tvm.device(str("cuda"), 0)
## building runtime
debug_g_mod = graph_runtime.GraphModuleDebug(
    compiled_graph_lib["debug_create"]("default", dev),
    [dev],
    compiled_graph_lib.get_graph_json(),
    "."
)

In [23]:
print(debug_g_mod.benchmark(dev))
times = debug_g_mod.run_individual(10, 3, 1000)
for idx, node in enumerate(debug_g_mod.debug_datum._nodes_list):
    print(f"{node['op']} : {times[idx]}")

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   0.0249       0.0249       0.0262       0.0239       0.0008                  


[07:42:02] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:65: Op #0 reshape_nop:
[07:42:02] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 0: 6.58997 us/iter
[07:42:02] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 1: 6.60019 us/iter
[07:42:02] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 2: 6.59817 us/iter
[07:42:07] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:65: Op #1 tvmgen_default_fused_nn_dense_add_nn_relu:
[07:42:07] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 0: 10.4939 us/iter
[07:42:07] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 1: 10.5464 us/iter
[07:42:07] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 2: 10.5339 us/iter
[07:42:12] /root/wang/tvm/src/runtime/graph_executor/deb

param : [0.0, 0.0, 0.0]
__nop : [6.5899669923171775e-06, 6.600192169541711e-06, 6.5981694839765936e-06]
param : [0.0, 0.0, 0.0]
param : [0.0, 0.0, 0.0]
tvmgen_default_fused_nn_dense_add_nn_relu : [1.0493895028803072e-05, 1.0546376382216401e-05, 1.053394240121245e-05]
param : [0.0, 0.0, 0.0]
param : [0.0, 0.0, 0.0]
tvmgen_default_fused_nn_dense_add_nn_relu_1 : [1.0481420089289032e-05, 1.0500668382620581e-05, 1.0496030181475126e-05]
param : [0.0, 0.0, 0.0]
param : [0.0, 0.0, 0.0]
tvmgen_default_fused_nn_dense_add : [1.0560148774250742e-05, 1.0543311405093934e-05, 1.0568597013195131e-05]
tvmgen_default_fused_nn_softmax : [1.0456835184123057e-05, 1.0474456607215662e-05, 1.0496131204522056e-05]


[07:42:23] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:65: Op #4 tvmgen_default_fused_nn_softmax:
[07:42:23] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 0: 10.4568 us/iter
[07:42:23] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 1: 10.4745 us/iter
[07:42:23] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 2: 10.4961 us/iter


In [26]:
loops = []

def find_width8(op):
    """Find all the 'tir.For' nodes whose extent can be divided by 8."""
    if isinstance(op, tvm.tir.For):
        if isinstance(op.extent, tvm.tir.IntImm):
            if op.extent.value % 8 == 0:
                loops.append(op)


def vectorize8(op):
    """Split can vectorize the loops found in `find_width8`."""
    if op in loops:
        extent = op.extent.value
        name = op.loop_var.name
        lo, li = te.var(name + ".outer"), te.var(name + ".inner")
        body = tvm.tir.stmt_functor.substitute(op.body, {op.loop_var: lo * 8 + li})
        body = tvm.tir.For(li, 0, 8, tvm.tir.ForKind.VECTORIZED, body)
        body = tvm.tir.For(lo, 0, extent // 8, tvm.tir.ForKind.SERIAL, body)
        return body
    return None


@tvm.tir.transform.prim_func_pass(opt_level=0)
def vectorize(f, mod, ctx):
    global loops

    tvm.tir.stmt_functor.post_order_visit(f.body, find_width8)

    if not loops:
        return f

    # The last list arugment indicates what kinds of nodes will be transformed.
    # Thus, in this case only `For` nodes will call `vectorize8`
    return f.with_body(tvm.tir.stmt_functor.ir_transform(f.body, None, vectorize8, ["tir.For"]))

with tvm.transform.PassContext( opt_level=3, config={"tir.add_lower_pass": [(2, vectorize)]}):
    lib = relay.build_module.build(net, target="cuda", target_host="llvm",params=params)

dev = tvm.device(str("cuda"), 0)
## building runtime
debug_g_mod = graph_runtime.GraphModuleDebug(
    compiled_graph_lib["debug_create"]("default", dev),
    [dev],
    compiled_graph_lib.get_graph_json(),
    "."
)
print(debug_g_mod.benchmark(dev))
times = debug_g_mod.run_individual(10, 3, 1000)
for idx, node in enumerate(debug_g_mod.debug_datum._nodes_list):
    print(f"{node['op']} : {times[idx]}")

Execution time summary:
 mean (ms)   median (ms)    max (ms)     min (ms)     std (ms)  
   0.0167       0.0167       0.0170       0.0164       0.0002                  


[07:54:47] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:65: Op #0 reshape_nop:
[07:54:47] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 0: 6.53562 us/iter
[07:54:47] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 1: 6.5358 us/iter
[07:54:47] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 2: 6.53944 us/iter
[07:54:52] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:65: Op #1 tvmgen_default_fused_nn_dense_add_nn_relu:
[07:54:52] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 0: 10.4723 us/iter
[07:54:52] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 1: 10.4692 us/iter
[07:54:52] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 2: 10.4641 us/iter
[07:54:58] /root/wang/tvm/src/runtime/graph_executor/debu

param : [0.0, 0.0, 0.0]
__nop : [6.535616651231553e-06, 6.53580333823292e-06, 6.539438057044784e-06]
param : [0.0, 0.0, 0.0]
param : [0.0, 0.0, 0.0]
tvmgen_default_fused_nn_dense_add_nn_relu : [1.0472294958008309e-05, 1.0469202060731043e-05, 1.0464139969813802e-05]
param : [0.0, 0.0, 0.0]
param : [0.0, 0.0, 0.0]
tvmgen_default_fused_nn_dense_add_nn_relu_1 : [1.0465960616506995e-05, 1.0456592616346655e-05, 1.0468039410216325e-05]
param : [0.0, 0.0, 0.0]
param : [0.0, 0.0, 0.0]
tvmgen_default_fused_nn_dense_add : [1.047626663325679e-05, 1.047342061289671e-05, 1.0473626966258518e-05]
tvmgen_default_fused_nn_softmax : [1.0446207952913207e-05, 1.0413013430963034e-05, 1.0427781216463869e-05]


[07:55:07] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:65: Op #4 tvmgen_default_fused_nn_softmax:
[07:55:07] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 0: 10.4462 us/iter
[07:55:07] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 1: 10.413 us/iter
[07:55:07] /root/wang/tvm/src/runtime/graph_executor/debug/graph_executor_debug.cc:68: Iteration: 2: 10.4278 us/iter


In [29]:
lower_to_te = tvm._ffi.get_global_func("relay.backend.LowerToTE")
cached_func = lower_to_te(net["main"])

dense is not optimized for this platform.
dense is not optimized for this platform.


dense is not optimized for this platform.


In [31]:
def _get_tensors(te_cached_func):
    return list(te_cached_func.inputs) + list(te_cached_func.outputs)

x = _get_tensors(cached_func)

In [32]:
x

[Tensor(shape=[1, 1, 28, 28], op.name=data),
 Tensor(shape=[128, 784], op.name=fc1_weight),
 Tensor(shape=[128], op.name=fc1_bias),
 Tensor(shape=[64, 128], op.name=fc2_weight),
 Tensor(shape=[64], op.name=fc2_bias),
 Tensor(shape=[1000, 64], op.name=fc3_weight),
 Tensor(shape=[1000], op.name=fc3_bias),
 Tensor(shape=[1, 1000], op.name=T_softmax_norm)]

In [42]:
sch = te.create_schedule([cached_func.outputs[0].op])

In [55]:
sch = sch.normalize()

schedule(0x5642f4bdee10)